In [5]:
import spacy
import re
import pandas as pd
from spacy.tokens import Span
from docx import Document  # 用于读取docx文件
from collections import defaultdict

In [6]:

# 1. 加载spaCy模型与自定义实体标签
nlp = spacy.load("en_core_web_sm")
ner = nlp.get_pipe("ner")
for label in ["POLICY", "LEGAL", "TIME_BASE", "TIME_2017", "PARAMETER", "AREA", "TARGET"]:
    ner.add_label(label)

In [7]:

# 2. 读取《2017政策.docx》全文档文本（核心：加载所有段落，排除图表描述）
def load_full_policy_doc(doc_path):
    doc = Document(doc_path)
    full_text = []
    # 排除图表/来源标注的关键词
    exclude_keywords = ["Figure", "Source:", "img", "---"]
    for paragraph in doc.paragraphs:
        para_text = paragraph.text.strip()
        # 跳过空段落和图表标注
        if not para_text or any(keyword in para_text for keyword in exclude_keywords):
            continue
        full_text.append(para_text)
    # 合并为全文档文本，按章节分隔符拆分模块
    full_text_str = "\n".join(full_text)
    # 按政策模块拆分（基于文档子标题）
    module_keywords = [
        "Congestion Impact Fee System",
        "Transportation Demand Management Policy for Companies",
        "Congestion Charging at Namsan Tunnel 1 and 3",
        "Parking Lot Restrictions for Facilities in Certain Areas",
        "Urban Traffic Improvement Promotion Act"
    ]
    # 拆分模块并建立{模块名: 文本}字典
    modules = defaultdict(str)
    current_module = "General"  # 初始模块（TDM基础定义）
    for line in full_text_str.split("\n"):
        # 检查是否切换模块
        module_match = next((kw for kw in module_keywords if kw in line), None)
        if module_match:
            current_module = module_match
        modules[current_module] += line + " "
    return modules

In [8]:

# 3. 全文档文本预处理（单模块处理函数）
def preprocess_module_text(module_text):
    # 1. 清洗特殊符号与冗余内容
    module_text = re.sub(r'\s+', ' ', module_text.strip())  # 统一空格
    module_text = re.sub(r'\(.*?\)', '', module_text)       # 去除括号内注释
    module_text = re.sub(r'[^\x00-\x7F]+', '', module_text) # 去除非ASCII乱码
    # 2. 表格文本结构化（以Table 1为例，其他表格类似）
    table1_pattern = r"Table 1. SMG, Ordinance on the Congestion Impact Fee Discount.*?2020 ~.*?congestion coefficient"
    table1_match = re.search(table1_pattern, module_text, re.DOTALL)
    if table1_match:
        table1_text = table1_match.group()
        # 提取2017年折扣规则
        table1_2017 = re.search(r'2017\s*× congestion coefficient', table1_text).group()
        # 替换表格文本为结构化描述
        module_text = module_text.replace(table1_text, f"2017年拥堵影响费折扣规则：{table1_2017}")
    return module_text

In [9]:

# 4. 多维度关键信息抽取函数（单模块）
def extract_module_info(module_name, module_text):
    info = {
        "政策大类": module_name,
        "法律依据": "",
        "基础实施时间": "",
        "2017年调整内容": "",
        "适用区域": "",
        "核心参数（2017年）": "",
        "影响对象": ""
    }
    doc = nlp(module_text)

    # 4.1 通用信息抽取（法律依据、基础时间）
    # 法律依据（匹配Act/Ordinance）
    legal_pattern = r'([A-Z][a-z\s]+Act|[A-Z][a-z\s]+Ordinance)'
    legal_matches = re.findall(legal_pattern, module_text)
    info["法律依据"] = ", ".join(list(set(legal_matches))) if legal_matches else "无明确记录"

    # 基础实施时间（匹配introduced/launched in 年份）
    time_base_pattern = r'(introduced|launched|enacted) in (\d{4})'
    time_base_match = re.search(time_base_pattern, module_text)
    if time_base_match:
        info["基础实施时间"] = time_base_match.group(2)

    # 4.2 模块专属信息抽取
    if "Congestion Impact Fee System" in module_name:
        # 核心参数：单位费、系数、门槛
        unit_fee = re.search(r'unit congestion impact fee is (\d+ to \d+) Korean won per m²', module_text)
        coeff_range = re.search(r'congestion coefficient varies from (\d+\.\d+) for (\w+) to (\d+\.\d+) for (\w+)', module_text)
        threshold = re.search(r'total floor area of (\d+) m² or more', module_text)
        params = []
        if unit_fee: params.append(f"单位费：{unit_fee.group(1)}韩元/㎡")
        if coeff_range: params.append(f"系数范围：{coeff_range.group(1)}（{coeff_range.group(2)}）-{coeff_range.group(3)}（{coeff_range.group(4)}）")
        if threshold: params.append(f"收费门槛：{threshold.group(1)}㎡以上")
        info["核心参数（2017年）"] = "; ".join(params)

        # 2017年调整：折扣规则
        adjust_2017 = re.search(r'2017年拥堵影响费折扣规则：(.*?) ', module_text)
        if adjust_2017:
            info["2017年调整内容"] = f"超3000㎡/30000㎡设施收费计算优化：{adjust_2017.group(1)}"

        # 影响对象
        info["影响对象"] = "1000㎡以上设施业主"

    elif "Transportation Demand Management Policy for Companies" in module_name:
        # 核心参数：折扣比例（Table 2）
        discount_pattern = r'Discount Rate.*?(\d+)%'
        discount_matches = list(set(re.findall(discount_pattern, module_text)))
        info["核心参数（2017年）"] = f"折扣比例：{', '.join(discount_matches)}%（多措施可叠加）"

        # 2017年调整：参与流程优化
        info["2017年调整内容"] = "维持折扣比例，简化中小企业参与流程，支持多措施叠加"

        # 影响对象
        info["影响对象"] = "1000㎡以上建筑企业、设施员工及使用者"

    elif "Namsan Tunnel" in module_name:
        # 核心参数：收费标准、时段、适用车辆
        fee = re.search(r'levy of KRW (\d+,?\d+)', module_text)
        time_slot = re.search(r'from (\d+:\d+ – \d+:\d+) Monday to Friday', module_text)
        vehicle = re.search(r'vehicles with only (\d+ or \d+) occupants', module_text)
        params = []
        if fee: params.append(f"收费标准：{fee.group(1)}韩元/次")
        if time_slot: params.append(f"收费时段：{time_slot.group(1)}（周1-周5）")
        if vehicle: params.append(f"适用车辆：{vehicle.group(1)}人车辆")
        info["核心参数（2017年）"] = "; ".join(params)

        # 2017年调整：无新增，系统优化
        info["2017年调整内容"] = "维持收费标准，优化收费系统响应速度"

        # 适用区域
        info["适用区域"] = "Namsan Tunnel 1 & 3"

        # 影响对象
        info["影响对象"] = "隧道通行1-2人私家车车主"

    elif "Parking Lot Restrictions" in module_name:
        # 核心参数：限制比例、适用区域类型
        limit_ratio = re.search(r'limited to (\d+)% of the parking lots in non-congested areas', module_text)
        area_type = re.search(r'expanded to ‘commercial areas and quasi residential areas’', module_text)
        params = []
        if limit_ratio: params.append(f"限制比例：拥堵区域为非拥堵区域{limit_ratio.group(1)}%")
        if area_type: params.append(f"区域类型：{area_type.group(1)}")
        info["核心参数（2017年）"] = "; ".join(params)

        # 2017年调整：延续2009年范围
        info["2017年调整内容"] = "延续2009年修订的管控范围（10个Class 1区域），无新增区域"

        # 适用区域
        area_size = re.search(r'(\d+\.\d+)km²', module_text)
        info["适用区域"] = f"首尔10个Class 1区域（{area_size.group(1)}km²）" if area_size else "首尔Class 1区域"

        # 影响对象
        info["影响对象"] = "拥堵区域商业/办公设施业主"

    return info

In [10]:

# 5. 全文档处理主函数
def process_full_policy_doc(doc_path):
    # 步骤1：加载并拆分模块
    modules = load_full_policy_doc(doc_path)
    # 步骤2：逐模块预处理与信息抽取
    full_info_list = []
    policy_id = 1
    for module_name, module_text in modules.items():
        # 跳过空模块
        if len(module_text.strip()) < 100:
            continue
        # 预处理
        processed_text = preprocess_module_text(module_text)
        # 抽取信息
        module_info = extract_module_info(module_name, processed_text)
        # 添加政策ID
        module_info["政策ID"] = f"TDM-2017-{policy_id:03d}"
        full_info_list.append(module_info)
        policy_id += 1
    # 转换为DataFrame
    full_info_df = pd.DataFrame(full_info_list)
    # 调整列顺序
    col_order = ["政策ID", "政策大类", "法律依据", "基础实施时间", "2017年调整内容", "适用区域", "核心参数（2017年）", "影响对象"]
    full_info_df = full_info_df[col_order]
    return full_info_df

In [12]:

# 6. 执行全文档处理（替换为你的docx文件路径）
if __name__ == "__main__":
    doc_path = "2017政策.docx"  # 你的文件路径
    full_policy_info = process_full_policy_doc(doc_path)
    # 输出结果
    print("《2017政策.docx》全文档关键信息抽取结果：")
    print("="*120)
    print(full_policy_info.to_string(index=False))
    # 保存为Excel（便于后续关联分析）
    full_policy_info.to_excel("2017政策全文档NLP抽取结果.xlsx", index=False)
    print("\n结果已保存至：2017政策全文档NLP抽取结果.xlsx")

《2017政策.docx》全文档关键信息抽取结果：
        政策ID                                                     政策大类                                                               法律依据 基础实施时间                          2017年调整内容                适用区域                    核心参数（2017年）                 影响对象
TDM-2017-001                                                  General                                                              无明确记录                                                                                                                  
TDM-2017-002                  Urban Traffic Improvement Promotion Act                                                      Promotion Act                                                                                                                  
TDM-2017-003                             Congestion Impact Fee System Promotion Act, Transport acknowledged it necessary to levy the Act   1990                                                                              